In [1]:
import sys
sys.path.insert(0, '../gdaxtrader/')

In [2]:
from datetime import datetime, timedelta

In [3]:
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

In [4]:
import pandas as pd
import psycopg2

In [18]:
import imp

In [35]:
import marketdata as mktdata
import visualize as vis
import scrape
import httpapi
import logger

imp.reload(scrape)
imp.reload(httpapi)

<module 'httpapi' from '../gdaxtrader/httpapi.py'>

In [29]:
def toiso(unix):
    return datetime.utcfromtimestamp(unix).isoformat()

In [30]:
def iso_start_end(rates):
    return toiso(rates[len(rates)-1][0]), toiso(rates[0][0])

In [36]:
end = datetime.utcnow() - timedelta(weeks=2)
start = end - timedelta(weeks=2)
rates = scrape.get_rates('BTC-USD', start, end, 60)

2017-12-03 23:57:11 INFO     getting HISTORIC RATES
2017-12-03 23:57:11 INFO     performing GET request
2017-12-03 23:57:11 INFO     url: https://api.gdax.com/products/BTC-USD/candles
2017-12-03 23:57:11 INFO     params:
2017-12-03 23:57:11 INFO     {'start': '2017-11-20T01:37:11.697929', 'end': '2017-11-20T04:57:11.697929', 'granularity': 60}
2017-12-03 23:57:11 INFO     executing AUTHENTICATED request
2017-12-03 23:57:11 INFO     unix timestamp: 1512363431.70833
2017-12-03 23:57:11 INFO     UTC ISO8601: 2017-12-04T04:57:11.708330
2017-12-03 23:57:11 INFO     GET request completed in 0.114433s.
2017-12-03 23:57:11 INFO     status code: 200
2017-12-03 23:57:11 INFO     reason: OK
2017-12-03 23:57:12 INFO     getting HISTORIC RATES
2017-12-03 23:57:12 INFO     performing GET request
2017-12-03 23:57:12 INFO     url: https://api.gdax.com/products/BTC-USD/candles
2017-12-03 23:57:12 INFO     params:
2017-12-03 23:57:12 INFO     {'start': '2017-11-19T22:16:00', 'end': '2017-11-20T01:36:00'

2017-12-03 23:57:23 INFO     url: https://api.gdax.com/products/BTC-USD/candles
2017-12-03 23:57:23 INFO     params:
2017-12-03 23:57:23 INFO     {'start': '2017-11-18T09:25:00', 'end': '2017-11-18T12:45:00', 'granularity': 60}
2017-12-03 23:57:23 INFO     executing AUTHENTICATED request
2017-12-03 23:57:23 INFO     unix timestamp: 1512363443.1493208
2017-12-03 23:57:23 INFO     UTC ISO8601: 2017-12-04T04:57:23.149321
2017-12-03 23:57:23 INFO     GET request completed in 0.144845s.
2017-12-03 23:57:23 INFO     status code: 200
2017-12-03 23:57:23 INFO     reason: OK
2017-12-03 23:57:24 INFO     getting HISTORIC RATES
2017-12-03 23:57:24 INFO     performing GET request
2017-12-03 23:57:24 INFO     url: https://api.gdax.com/products/BTC-USD/candles
2017-12-03 23:57:24 INFO     params:
2017-12-03 23:57:24 INFO     {'start': '2017-11-18T06:04:00', 'end': '2017-11-18T09:24:00', 'granularity': 60}
2017-12-03 23:57:24 INFO     executing AUTHENTICATED request
2017-12-03 23:57:24 INFO     unix 

2017-12-03 23:57:34 INFO     params:
2017-12-03 23:57:34 INFO     {'start': '2017-11-16T17:13:00', 'end': '2017-11-16T20:33:00', 'granularity': 60}
2017-12-03 23:57:34 INFO     executing AUTHENTICATED request
2017-12-03 23:57:34 INFO     unix timestamp: 1512363454.555862
2017-12-03 23:57:34 INFO     UTC ISO8601: 2017-12-04T04:57:34.555862
2017-12-03 23:57:34 INFO     GET request completed in 0.14808s.
2017-12-03 23:57:34 INFO     status code: 200
2017-12-03 23:57:34 INFO     reason: OK
2017-12-03 23:57:35 INFO     getting HISTORIC RATES
2017-12-03 23:57:35 INFO     performing GET request
2017-12-03 23:57:35 INFO     url: https://api.gdax.com/products/BTC-USD/candles
2017-12-03 23:57:35 INFO     params:
2017-12-03 23:57:35 INFO     {'start': '2017-11-16T13:52:00', 'end': '2017-11-16T17:12:00', 'granularity': 60}
2017-12-03 23:57:35 INFO     executing AUTHENTICATED request
2017-12-03 23:57:35 INFO     unix timestamp: 1512363455.530535
2017-12-03 23:57:35 INFO     UTC ISO8601: 2017-12-04T

2017-12-03 23:57:46 INFO     executing AUTHENTICATED request
2017-12-03 23:57:46 INFO     unix timestamp: 1512363466.003755
2017-12-03 23:57:46 INFO     UTC ISO8601: 2017-12-04T04:57:46.003755
2017-12-03 23:57:46 INFO     GET request completed in 0.150896s.
2017-12-03 23:57:46 INFO     status code: 200
2017-12-03 23:57:46 INFO     reason: OK
2017-12-03 23:57:46 INFO     getting HISTORIC RATES
2017-12-03 23:57:46 INFO     performing GET request
2017-12-03 23:57:46 INFO     url: https://api.gdax.com/products/BTC-USD/candles
2017-12-03 23:57:46 INFO     params:
2017-12-03 23:57:46 INFO     {'start': '2017-11-14T21:40:00', 'end': '2017-11-15T01:00:00', 'granularity': 60}
2017-12-03 23:57:46 INFO     executing AUTHENTICATED request
2017-12-03 23:57:46 INFO     unix timestamp: 1512363466.985993
2017-12-03 23:57:46 INFO     UTC ISO8601: 2017-12-04T04:57:46.985993
2017-12-03 23:57:47 INFO     GET request completed in 0.165921s.
2017-12-03 23:57:47 INFO     status code: 200
2017-12-03 23:57:47 

2017-12-03 23:57:58 INFO     UTC ISO8601: 2017-12-04T04:57:58.909231
2017-12-03 23:57:59 INFO     GET request completed in 0.118913s.
2017-12-03 23:57:59 INFO     status code: 200
2017-12-03 23:57:59 INFO     reason: OK
2017-12-03 23:57:59 INFO     getting HISTORIC RATES
2017-12-03 23:57:59 INFO     performing GET request
2017-12-03 23:57:59 INFO     url: https://api.gdax.com/products/BTC-USD/candles
2017-12-03 23:57:59 INFO     params:
2017-12-03 23:57:59 INFO     {'start': '2017-11-13T05:28:00', 'end': '2017-11-13T08:48:00', 'granularity': 60}
2017-12-03 23:57:59 INFO     executing AUTHENTICATED request
2017-12-03 23:57:59 INFO     unix timestamp: 1512363479.855724
2017-12-03 23:57:59 INFO     UTC ISO8601: 2017-12-04T04:57:59.855724
2017-12-03 23:57:59 INFO     GET request completed in 0.127789s.
2017-12-03 23:57:59 INFO     status code: 200
2017-12-03 23:57:59 INFO     reason: OK
2017-12-03 23:58:00 INFO     getting HISTORIC RATES
2017-12-03 23:58:00 INFO     performing GET request


2017-12-03 23:58:10 INFO     status code: 200
2017-12-03 23:58:10 INFO     reason: OK
2017-12-03 23:58:11 INFO     getting HISTORIC RATES
2017-12-03 23:58:11 INFO     performing GET request
2017-12-03 23:58:11 INFO     url: https://api.gdax.com/products/BTC-USD/candles
2017-12-03 23:58:11 INFO     params:
2017-12-03 23:58:11 INFO     {'start': '2017-11-11T13:16:00', 'end': '2017-11-11T16:36:00', 'granularity': 60}
2017-12-03 23:58:11 INFO     executing AUTHENTICATED request
2017-12-03 23:58:11 INFO     unix timestamp: 1512363491.328135
2017-12-03 23:58:11 INFO     UTC ISO8601: 2017-12-04T04:58:11.328135
2017-12-03 23:58:11 INFO     GET request completed in 0.099465s.
2017-12-03 23:58:11 INFO     status code: 200
2017-12-03 23:58:11 INFO     reason: OK
2017-12-03 23:58:12 INFO     getting HISTORIC RATES
2017-12-03 23:58:12 INFO     performing GET request
2017-12-03 23:58:12 INFO     url: https://api.gdax.com/products/BTC-USD/candles
2017-12-03 23:58:12 INFO     params:
2017-12-03 23:58:

2017-12-03 23:58:22 INFO     getting HISTORIC RATES
2017-12-03 23:58:22 INFO     performing GET request
2017-12-03 23:58:22 INFO     url: https://api.gdax.com/products/BTC-USD/candles
2017-12-03 23:58:22 INFO     params:
2017-12-03 23:58:22 INFO     {'start': '2017-11-09T21:04:00', 'end': '2017-11-10T00:24:00', 'granularity': 60}
2017-12-03 23:58:22 INFO     executing AUTHENTICATED request
2017-12-03 23:58:22 INFO     unix timestamp: 1512363502.631465
2017-12-03 23:58:22 INFO     UTC ISO8601: 2017-12-04T04:58:22.631465
2017-12-03 23:58:22 INFO     GET request completed in 0.140897s.
2017-12-03 23:58:22 INFO     status code: 200
2017-12-03 23:58:22 INFO     reason: OK
2017-12-03 23:58:23 INFO     getting HISTORIC RATES
2017-12-03 23:58:23 INFO     performing GET request
2017-12-03 23:58:23 INFO     url: https://api.gdax.com/products/BTC-USD/candles
2017-12-03 23:58:23 INFO     params:
2017-12-03 23:58:23 INFO     {'start': '2017-11-09T17:43:00', 'end': '2017-11-09T21:03:00', 'granularit

2017-12-03 23:58:33 INFO     url: https://api.gdax.com/products/BTC-USD/candles
2017-12-03 23:58:33 INFO     params:
2017-12-03 23:58:33 INFO     {'start': '2017-11-08T04:52:00', 'end': '2017-11-08T08:12:00', 'granularity': 60}
2017-12-03 23:58:33 INFO     executing AUTHENTICATED request
2017-12-03 23:58:33 INFO     unix timestamp: 1512363513.936494
2017-12-03 23:58:33 INFO     UTC ISO8601: 2017-12-04T04:58:33.936494
2017-12-03 23:58:34 INFO     GET request completed in 0.100862s.
2017-12-03 23:58:34 INFO     status code: 200
2017-12-03 23:58:34 INFO     reason: OK
2017-12-03 23:58:34 INFO     getting HISTORIC RATES
2017-12-03 23:58:34 INFO     performing GET request
2017-12-03 23:58:34 INFO     url: https://api.gdax.com/products/BTC-USD/candles
2017-12-03 23:58:34 INFO     params:
2017-12-03 23:58:34 INFO     {'start': '2017-11-08T01:31:00', 'end': '2017-11-08T04:51:00', 'granularity': 60}
2017-12-03 23:58:34 INFO     executing AUTHENTICATED request
2017-12-03 23:58:34 INFO     unix t

2017-12-03 23:58:45 INFO     params:
2017-12-03 23:58:45 INFO     {'start': '2017-11-06T12:40:00', 'end': '2017-11-06T16:00:00', 'granularity': 60}
2017-12-03 23:58:45 INFO     executing AUTHENTICATED request
2017-12-03 23:58:45 INFO     unix timestamp: 1512363525.2027001
2017-12-03 23:58:45 INFO     UTC ISO8601: 2017-12-04T04:58:45.202700
2017-12-03 23:58:45 INFO     GET request completed in 0.098358s.
2017-12-03 23:58:45 INFO     status code: 200
2017-12-03 23:58:45 INFO     reason: OK
2017-12-03 23:58:46 INFO     getting HISTORIC RATES
2017-12-03 23:58:46 INFO     performing GET request
2017-12-03 23:58:46 INFO     url: https://api.gdax.com/products/BTC-USD/candles
2017-12-03 23:58:46 INFO     params:
2017-12-03 23:58:46 INFO     {'start': '2017-11-06T09:19:00', 'end': '2017-11-06T12:39:00', 'granularity': 60}
2017-12-03 23:58:46 INFO     executing AUTHENTICATED request
2017-12-03 23:58:46 INFO     unix timestamp: 1512363526.129419
2017-12-03 23:58:46 INFO     UTC ISO8601: 2017-12-0

##### fig = vis.hist_rates(rates, product='BTC-USD', movavg_windows=[14,28])

In [ ]:
plotly.offline.iplot(fig)

In [37]:
len(rates)

20053

In [38]:
iso_start_end(rates)

('2017-11-06T04:57:00', '2017-11-20T04:57:00')

In [39]:
scrape.store_rates(rates, 'BTC-USD')

2017-12-03 23:59:17 INFO     storing 20053 HISTORIC RATES
2017-12-03 23:59:17 INFO     first rate date: 2017-11-06T04:57:00
2017-12-03 23:59:17 INFO     last rate date: 2017-11-20T04:57:00
2017-12-03 23:59:17 INFO     connecting to SQL DB
2017-12-03 23:59:17 INFO     params:
2017-12-03 23:59:17 INFO     {'dbname': 'gdaxtrader', 'user': 'root', 'host': 'localhost', 'port': 26257}
2017-12-03 23:59:17 INFO     connected to DB.
2017-12-03 23:59:17 INFO     creating database gdaxtrader (if necessary).
2017-12-03 23:59:17 INFO     creating table hist_rates (if necessary).
2017-12-03 23:59:17 INFO     inserting rates into DB...
2017-12-03 23:59:17 WARNING  no new rates for current batch.
2017-12-03 23:59:17 WARNING  no new rates for current batch.
2017-12-03 23:59:18 WARNING  no new rates for current batch.
2017-12-03 23:59:18 WARNING  no new rates for current batch.
2017-12-03 23:59:18 WARNING  no new rates for current batch.
2017-12-03 23:59:18 WARNING  no new rates for current batch.
2017-